In [32]:
import pandas as pd
from datetime import datetime, timedelta
import math
import numpy as np
from pandas.io.json import json_normalize

In [2]:
#the players number column, age, weight columns have NaN's in them probably can drop the number and weight columns
#as they are not features 
roster = pd.read_csv('stats_files/rosters', sep = '|')
roster.shape


/Users/MattBarlowe/.virtualenvs/prospectmodel/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,2,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(136182, 13)

In [3]:
#makes sure age matches this format and removes players which we don't have a birthdate for
#also excludes weird dates that might include dates over 31 or months over 12 or entries of 00
roster = roster[roster['Birthdate'].str.match('(\d\d\d\d)\-((?!00)0[1-9]|1[0-2])\-((?!00)(0[1-9]|[12]\d|3[01]))')]
roster.shape

(136021, 13)

In [4]:
#had two birthdates on september 31st just excluded them 
roster = roster[~roster['Birthdate'].str.match('(\d\d\d\d)\-09\-31')]
roster.shape

(136018, 13)

In [5]:
#Create the season start to calculate age at start of season for age adjusted statistics
roster['season_start'] = roster['season'].apply(lambda x: x - 1).apply('09-15-{}'.format)
roster['Birthdate'] = pd.to_datetime(roster['Birthdate'], format='%Y-%m-%d')
roster['season_start'] = pd.to_datetime(roster['season_start'], format='%m-%d-%Y')
roster['draft_cutoff'] = roster['season'].apply('09-15-{}'.format)
roster['draft_cutoff'] = pd.to_datetime(roster['draft_cutoff'], format='%m-%d-%Y')
#EP lists players Age as their current age instead of the age they were at the season
#I'm calculating their age as the age they are at the draft cutoff of that season
#in order to see if they are draft eligible that season. I have to calculate the players
#at sept. 15th of the draft year for that respective season. I then use that draft
#cutoff age to determine the years from their first draft eligible year
roster['Age'] = (roster['season_start'] - roster['Birthdate'])
roster['Age'] = roster['Age'] / timedelta(days=365)
roster['Age'] = roster['Age'].round(decimals=2)
roster['draft_cutoff_age'] = roster['draft_cutoff'] - roster['Birthdate']
roster['draft_cutoff_age'] = roster['draft_cutoff_age'] / timedelta(days=365)
roster['draft_cutoff_age'] = roster['draft_cutoff_age'].round(decimals=2)

In [6]:
#Remove goalies from the dataframe
roster = roster[roster['Position'] != 'G']

In [7]:
#this function determines the years away from first draft eligible year for the season
#of the row i then apply it to the draft_cutoff_age column using pandas .apply() function
def draft_year(age):
    
    if age >=18 and age < 19:
        return 0
    else:
        return math.floor(age-18)
        
        
roster['draft_year'] = roster['draft_cutoff_age'].apply(draft_year)
#roster.Age.astype

In [8]:
stats = pd.read_csv('stats_files/player_stats', sep='|')
stats.head()
roster = roster.merge(stats[['player_id', 'season', 'GP', 'G', 'A', 'TP', 'PIM', 'playoff_GP',  
                           'playoff_G', 'playoff_A', 'playoff_TP', 'playoff_PIM', 'playoff_+/-']], 
                          on =['player_id', 'season'])

In [9]:
#converting height into decimal values of feet
def clean_height(value):
    '''
    Function to convert height meausered in feet and inches to a decimal
    value
    Input:
    value - height measured in feet and inches
    Output:
    decimal_value - height returned in feet and inches converted to a decimal
                    value of feet
    '''
    try:
        value = value.replace("'", '.').replace(' ', '').replace('"', '')
        value_list = value.split('.')
        value_list[1] = float(value_list[1])/12
        value = float(value_list[0]) + value_list[1]
        value = round(value, 2)
    except IndexError as ex:
        value = ''

    return value

roster['HT'] = roster['HT'].astype('str').apply(clean_height)


In [47]:
#joining league data to the dataframe
leagues_hist = pd.read_csv('stats_files/leagues_hist_table', sep='|')

roster = roster.merge(leagues_hist[['league', 'season', 'team_id']], on=['season', 'team_id'])

In [101]:
roster['Player'] = roster['Player'].str.strip()

,#,Player,Age,Position,Birthdate,Birthplace,HT,WT,Shots,player_id,...,A,TP,PIM,playoff_GP,playoff_G,playoff_A,playoff_TP,playoff_PIM,playoff_+/-,league
148343,13,Nico Hischier,17.71,C,1999-01-04,"Brig, SUI",6.08,174,L,116244,...,48,86,24,6,3,4,7,0,2,QMJHL


In [117]:
import bs4 
import requests
import re
import pandas as pd

2003
['#292', 'New Jersey Devils', 'Arseni Bondarev (F)', '', '', '', '', '', '']
2004
['#291', 'Philadelphia Flyers', 'John Carter (F)', '', '', '', '', '', '']
2005
['#230', 'Nashville Predators', 'Patric Hörnqvist (F)', '10', '649', '203', '208', '411', '333']
2006
['#213', 'Carolina Hurricanes', 'Justin Krueger (D)', '', '', '', '', '', '']
2007
['#211', 'Columbus Blue Jackets', 'Trent Vogelhuber (F)', '', '', '', '', '', '']
2008
['#211', 'Detroit Red Wings', 'Jesper Samuelsson (F)', '', '', '', '', '', '']
2009
['#211', 'Montréal Canadiens', 'Petteri Similä  (G)', '', '', '', '', '', '']
2010
['#210', 'Boston Bruins', 'Zach Trotman (D)', '4', '70', '3', '9', '12', '22']
2011
['#211', 'Chicago Blackhawks', 'Johan Mattsson (G)', '', '', '', '', '', '']
2012
['#211', 'Los Angeles Kings', 'Nick Ebert (D)', '', '', '', '', '', '']
2013
['#211', 'Chicago Blackhawks', 'Robin Press (D)', '', '', '', '', '', '']
2014
['#210', 'Los Angeles Kings', 'Jacob Middleton (D)', '', '', '', '', '',

['#292', 'New Jersey Devils', 'Arseni Bondarev (F)', '', '', '', '', '', '']

2628

['\n                            #217\n                        ',
 '\nPittsburgh Penguins\n',
 '\n  \n\nWilliam Reilly (D)\n\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n']

9

8

9